In [5]:
import torch
from torch import Tensor
import torch.nn as nn
from typing import Type, Any, Callable, Union, List, Optional
import os
import pprint

# [需求]
在torch1.12里面有个EfficientNetV2网络，你把他的代码分离出来。然后改写成我下面的形式。
仿照我下面代码的resnet50改。改完了用一个(1,3,224,224)的向量测试下，能不能正常推理。
保证"efficientnet_v2_s","efficientnet_b0","efficientnet_b2","efficientnet_b3","efficientnet_v2_m"能使用。
    

网络还是很大的话，用这个代替普通的conv
--------------------------------------------------------------
```python3
暂时没用
class dpconv(nn.Module):
    def __init__(self,in_ch,out_ch,ks=3,st=1,pd=1,gp=1,dn=1):
        super(dpconv, self).__init__()
        # 也相当于分组为1的分组卷积
        self.conv3x3 = nn.Sequential()
        self.conv3x3.add_module("conv3x1",nn.Conv2d(in_ch, in_ch, kernel_size=[ks,1], stride=st,
                        padding=[dn,0], groups=gp, bias=False, dilation=dn))
        self.conv3x3.add_module("conv1x3",nn.Conv2d(in_ch, out_ch, kernel_size=[1,ks], stride=st,
                        padding=[0,dn], groups=gp, bias=False, dilation=dn))
    def forward(self,input):
        out = self.conv3x3(input)
        return out
def conv3x3_dp(in_planes: int, out_planes: int,kernelsize:int=3, stride: int = 1, groups: int = 1, dilation: int = 1) -> nn.Conv2d:
    return dpconv(in_planes,out_planes,ks=kernelsize,st=stride,pd=dilation,gp=groups,dn=dilation)

class dwconv(nn.Module):
    def __init__(self,in_ch,out_ch,ks=3,st=1,pd=1,gp=1,dn=1):
        super(dwconv, self).__init__()
        self.conv3x3 = nn.Sequential()
        self.conv3x3.add_module("dconv", nn.Conv2d(in_channels=in_ch,
                                    out_channels=in_ch,
                                    kernel_size=ks,
                                    stride=st,
                                    padding=pd,
                                    groups=in_ch))
        self.conv3x3.add_module("wconv",nn.Conv2d(in_channels=in_ch,
                                    out_channels=out_ch,
                                    kernel_size=1,
                                    stride=1,
                                    padding=0,
                                    groups=gp,dilation=dn))
    def forward(self,input):
        out = self.conv3x3(input)
        return out


def conv3x3_dw(in_planes: int, out_planes: int,kernelsize:int=3, stride: int = 1, groups: int = 1, dilation: int = 1) -> nn.Conv2d:
    """3x3 convolution with padding"""
    return  dwconv(in_planes,out_planes,ks=kernelsize,st=stride,pd=dilation,gp=groups,dn=dilation)

def choice_conv(flg):
    if flg ==1:
        return nn.Conv2d
    elif flg == 2:
        return conv3x3_dp
    elif flg ==3:
        return conv3x3_dw
    else:
        RuntimeError
```
---------------------------------

In [6]:

class dwconv(nn.Module):
    def __init__(self,in_ch,out_ch,ks=3,st=1,pd=1,gp=1,dn=1):
        super(dwconv, self).__init__()
        # 也相当于分组为1的分组卷积
        self.depth_conv = nn.Conv2d(in_channels=in_ch,
                                    out_channels=in_ch,
                                    kernel_size=ks,
                                    stride=st,
                                    padding=pd,
                                    groups=in_ch)
        self.point_conv = nn.Conv2d(in_channels=in_ch,
                                    out_channels=out_ch,
                                    kernel_size=1,
                                    stride=1,
                                    padding=0,
                                    groups=gp,dilation=dn)
    def forward(self,input):
        out = self.depth_conv(input)
        out = self.point_conv(out)
        return out


def conv3x3(in_planes: int, out_planes: int, stride: int = 1, groups: int = 1, dilation: int = 1) -> nn.Conv2d:
    """3x3 convolution with padding"""
    return  dwconv(in_planes,out_planes,ks=3,st=stride,pd=dilation,gp=groups,dn=dilation)


def conv1x1(in_planes: int, out_planes: int, stride: int = 1) -> nn.Conv2d:
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class BasicBlock(nn.Module):
    expansion: int = 1

    def __init__(
            self,
            inplanes: int,
            planes: int,
            stride: int = 1,
            downsample: Optional[nn.Module] = None,
            groups: int = 1,
            base_width: int = 64,
            dilation: int = 1,
            norm_layer: Optional[Callable[..., nn.Module]] = None
    ) -> None:
        super(BasicBlock, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError('BasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x: Tensor) -> Tensor:
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    expansion: int = 4
    def __init__(
            self,
            inplanes: int,
            planes: int,
            stride: int = 1,
            downsample: Optional[nn.Module] = None,
            groups: int = 1,
            base_width: int = 64,
            dilation: int = 1,
            norm_layer: Optional[Callable[..., nn.Module]] = None
    ) -> None:
        super(Bottleneck, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        width = int(planes * (base_width / 64.)) * groups
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x: Tensor) -> Tensor:
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class ResNet(nn.Module):
    def __init__(
            self,
            block: Type[Union[BasicBlock, Bottleneck]],
            layers: List[int],
            num_classes1: int = 1000,
            num_classes2: int = 1000,
            num_classes3: int = 1000,
            zero_init_residual: bool = False,
            groups: int = 1,
            width_per_group: int = 64,
            replace_stride_with_dilation: Optional[List[bool]] = None,
            norm_layer: Optional[Callable[..., nn.Module]] = None
    ) -> None:
        super(ResNet, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None "
                             "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2,
                                       dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2,
                                       dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2,
                                       dilate=replace_stride_with_dilation[2])
        self.layer5 = self._make_banch_layer(block, 512, layers[3], stride=2,
                                             dilate=replace_stride_with_dilation[2])
        self.layer6 = self._make_banch_layer(block, 512, layers[3], stride=2,
                                             dilate=replace_stride_with_dilation[2])
        self.avgpool1 = nn.AdaptiveAvgPool2d((1, 1))
        self.avgpool2 = nn.AdaptiveAvgPool2d((1, 1))
        self.avgpool3 = nn.AdaptiveAvgPool2d((1, 1))
        self.fc1 = nn.Linear(512 * block.expansion, num_classes1)
        self.fc2 = nn.Linear(512 * block.expansion, num_classes2)
        self.fc3 = nn.Linear(512 * block.expansion, num_classes3)
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)  # type: ignore[arg-type]
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)  # type: ignore[arg-type]
    def get_name(self):
        return 'resnet50'
    def _make_layer(self, block: Type[Union[BasicBlock, Bottleneck]], planes: int, blocks: int,
                    stride: int = 1, dilate: bool = False) -> nn.Sequential:
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:  # TODO
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation, norm_layer))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer))

        return nn.Sequential(*layers)

    def _make_banch_layer(self, block: Type[Union[BasicBlock, Bottleneck]], planes: int, blocks: int,
                          stride: int = 1, dilate: bool = False) -> nn.Sequential:
        # for layer5
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:  # TODO
            downsample = nn.Sequential(
                conv1x1(1024, 2048, stride),
                norm_layer(2048),
            )

        layers = []
        layers.append(block(1024, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation, norm_layer))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer))

        return nn.Sequential(*layers)

    def _forward_impl(self, x: Tensor) -> Tensor:
        # See note [TorchScript super()]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)

        x1 = self.layer4(x)
        x1 = self.avgpool1(x1)
        x1 = torch.flatten(x1, 1)
        x1 = self.fc1(x1)

        x2 = self.layer5(x)
        x2 = self.avgpool2(x2)
        x2 = torch.flatten(x2, 1)
        x2 = self.fc2(x2)
        
        x3 = self.layer6(x)
        x3 = self.avgpool3(x3)
        x3 = torch.flatten(x3, 1)
        x3 = self.fc3(x3)
        return [x1, x2,x3]

    def forward(self, x: Tensor) -> Tensor:
        return self._forward_impl(x)


def _resnet(
        arch: str,
        block: Type[Union[BasicBlock, Bottleneck]],
        layers: List[int],
        pretrained: bool,
        progress: bool,
        **kwargs: Any
) -> ResNet:
    model = ResNet(block, layers, **kwargs)
    return model


def resnet50_th(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> ResNet:
    """base on resnet50"""
    return _resnet('resnet50', Bottleneck, [3, 4, 6, 3], pretrained, progress,
                   **kwargs)

In [9]:
if __name__ == '__main__':
    m = resnet50_th(num_classes1=2, num_classes2=3, num_classes3=4)
    pth = r'123.pth'
    d = torch.randn(1, 3, 224, 224)
    pprint.pprint(m(d))
    torch.save(m,pth)
    print(f"[{pth}]   {os.stat(pth).st_size/1024/1024:.4f} M")

[tensor([[-0.5004,  0.1439]], grad_fn=<AddmmBackward>),
 tensor([[0.2285, 0.5533, 0.3779]], grad_fn=<AddmmBackward>),
 tensor([[-0.1768,  0.2447,  0.8454, -0.0250]], grad_fn=<AddmmBackward>)]
[123.pth]   118.3983 M
